In [1]:
###################### Install all necessary libraries #########################

from google.colab import files
!pip install praw -q
!pip install yfinance --upgrade --no-cache-dir -q
!pip install yahoo_fin -q
!pip install yahoo_fin --upgrade -q
!pip install requests_html -q
!pip install mpld3 -q
!pip install fastapi -q
!pip install colabcode -q

################################################################################

     |████████████████████████████████| 163kB 6.6MB/s 
     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 6.3MB 7.2MB/s 
     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 92kB 4.4MB/s 
     |████████████████████████████████| 81kB 4.6MB/s 
     |████████████████████████████████| 81kB 4.1MB/s 
ERROR: pyppeteer 0.2.5 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 153kB 5.1MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 890kB 5.1MB/s 
     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 10.1MB 6.9MB/s 
     |████████████████████████████████| 

In [2]:
#########################################  DATA REQUESTS #############################################

# To provide current date and time in an format understandable by our API
from datetime import datetime 
from datetime import timedelta
# Properly display html docs before we could work on it
from lxml import html 
# Requests HTML data from websites and api 
import requests 
from requests import Request, Session

# Connecting with an API without any exceptions
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects 

# Clean the result from API's
import json  
from bs4 import BeautifulSoup

##########################################  DATA MINING ##############################################

# To get posts from reddit
import praw

# To get the current stock prices
import yfinance as yf  
from yahoo_fin.stock_info import get_data

# Connect to twitter API
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy

from urllib.request import urlopen

# Library to set up email alerts 
import smtplib

#######################################  DATA MANIPULATION ###########################################

import numpy as np
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import os
import time

#######################################  DATA VISUALIZATION ###########################################
 
import plotly.express as px 
import plotly.graph_objects as go
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter3d, Layout, Scene
from plotly.offline import iplot
import mpld3
import plotly

########################################### DATA TRANSFER ############################################

from fastapi import FastAPI
from colabcode import ColabCode 

#######################################################################################################

In [3]:
def CombinedAnalysis_Stocks(stocks, amt, time):
    """
    Get the line plots of top ```amt``` treding stocks for the last ```time``` days.  

    Parameters
    ----------
    stocks : pd.DataFrame
    amt    : int
    time   : int

    Returns
    -------
    Graphs
    """
    ######################## Get the Top ```amt``` values ######################

    try:
      top_stocks = stocks.sort_values('Number_Of_Occurences', ascending=False).head(amt)
    except:
      top_stocks = stocks
    
    ############################ Collect Data and Plot #########################
    
    # Create an empty data Frame to store retreived data
    data_stock = pd.DataFrame(columns=top_stocks.columns)

    for tick in top_stocks['Ticker']:
      tick = tick.replace(" ","")
      try:
        
        # Collect data from yahoo finance function get_data using the stock ticker
        # Use pd.concat to store all data in one DataFrame
        data_stock = pd.concat([get_data(tick,
                              datetime.now() - timedelta(days=time),
                              datetime.date(datetime.now()), 
                              index_as_date=True),data_stock],
                            axis = 0)
      
      except:
        pass
    
    # Plot the line graph of all the stocks togather
    fig = px.line(data_stock, 
                  x=data_stock.index, 
                  y=data_stock['close'], 
                  color="ticker")
    
    fig.update_traces(mode="markers+lines")
    fig.update_xaxes(title="Date")
    fig.update_layout(
        hoverlabel=dict(
          bgcolor="white",
          font_size=16,
          font_family="Rockwell"
          )
    ) 
    plotly.offline.plot(fig, filename='AnalyzeStocks.html')
    ret_Graph('./AnalyzeStocks.html')
    # Display result (graph)
    fig.show()
    
    ############################################################################

In [4]:
def Stocks_linePlots(stocks, amt, time):
    """
    Get the line plots of top ```amt``` treding stocks for the last ```time``` days.  

    Parameters
    ----------
    stocks : pd.DataFrame
    amt    : int
    time   : int

    Returns
    -------
    Graphs
    """
    ######################## Get the Top ```amt``` values ######################
    
    try:
      top_stocks = stocks.sort_values('Number_Of_Occurences', ascending=False).head(amt)
    except:
      top_stocks = stocks 
    
    ############################ Collect Data and Plot #########################
    
    for tick in top_stocks['Ticker']:
      tick = tick.replace(" ", "")
      try:
        # Collect data from yahoo finance function get_data using the stock ticker
        data_stock = get_data(tick,
                              datetime.now() - timedelta(days= time),
                              datetime.date(datetime.now()),
                              index_as_date=True)
        
        # Start plotting the line graph in plotly.express
        fig = px.line(data_stock, 
                      x=data_stock.index, 
                      y=data_stock['close'], 
                      color="ticker")
        
        fig.update_traces(mode="markers+lines")
        fig.update_layout(
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
                font_family="Rockwell"
            )
        ) 
        fig.update_xaxes(title= tick)
        plotly.offline.plot(fig, filename='IndividualLine.html')
        ret_Graph('./IndividualLine.html')
        # Display the result (graph)
        fig.show()
      except:
        pass
  
    ############################################################################

In [5]:
def Stocks_candlePlots(stocks, amt, time):
    """
    Get the plots of top ```amt``` treding stocks for the last ```time``` days.  

    Parameters
    ----------
    stocks : pd.DataFrame
    amt    : int
    time   : int

    Returns
    -------
    Graphs
    """
    ######################## Get the Top ```amt``` values ######################
    
    try:
      top_stocks = stocks.sort_values('Number_Of_Occurences', ascending=False).head(amt)
    except:
      top_stocks = stocks
    
    ############################ Collect Data and Plot #########################
    
    for tick in top_stocks['Ticker']:
      tick = tick.replace(" ", "")
      try:
        # Collect data from yahoo finance function get_data using the stock ticker
        df = get_data(tick,
                      datetime.now() - timedelta(days= time),
                      datetime.date(datetime.now()), 
                      index_as_date=True)
        
        # Start plotting the graphs in plotly.graph.objects
        fig = go.Figure(data=[go.Candlestick(x=df.index,
                  open=df['open'],
                  high=df['high'],
                  low=df['low'],
                  close=df['close'])])
        
        fig.update_layout(
          yaxis_title=tick,
          xaxis_title = 'Date'
        )
        plotly.offline.plot(fig, filename='IndividualAnalysis.html')
        ret_Graph('./IndividualAnalysis.html')
        # Display the result
        fig.show()

      except:
        pass

    ############################################################################

In [6]:
def Popular_Crypto(crypto_data, amt):
    """
    Get the growth rate of most popular ```amt``` crypto's in the last 24h and plot their graphs

    Parameters
    ----------
    crypto_data : pd.DataFrame
    amt         : int

    Returns
    -------
    Graph
    """
    ############################ Collect Data Req ##############################
    
    popular_crypto_data = crypto_data.sort_values('volume_24', ascending=False).head(amt)
    
    ################################## Plot 1 ##################################

    ax = plt.subplots(figsize=(13,7))
    sns.set(style="whitegrid", color_codes=True)
    sns.barplot(data=popular_crypto_data,
                x=popular_crypto_data['Name'],
                y=popular_crypto_data['percent_change_24h'],
                palette="Blues_d", 
                label = "percent_change_24h")
    plt.xticks(rotation='vertical')
    plt.title('Most Popular Crypto')
    
    # Display the result
    plt.show() 
    
    ############################################################################

In [7]:
def Growing_Crypto(crypto_data, amt, period):
    """
    Get the top ```amt``` crypto with maximum growth in the last 24h and plot their graphs

    Parameters
    ----------
    crypto_data : pd.DataFrame
    amt         : int
    period      : int

    Returns
    -------
    Graphs
    """
    ########################### Collect Data Req ###############################

    top_crypto_data = crypto_data.sort_values('percent_change_24h'if period == 24 else 'percent_change_7d', ascending=False).head(amt)
    
    ################################# Plot 1 ###################################
    f, ax = plt.subplots(figsize=(13,7))

    if period == 24:
      sns.barplot(data=top_crypto_data,
                  x=top_crypto_data['Name'],
                  y=top_crypto_data['percent_change_24h'],
                  palette="gist_earth",#YlOrRd_r
                  label = "percent_change_24h")
    else:
      sns.barplot(data=top_crypto_data,
                  x=top_crypto_data['Name'],
                  y=top_crypto_data['percent_change_7d'],
                  palette="gist_earth",#YlOrRd_r
                  label = "percent_change_7d")
    
    plt.xticks(rotation='vertical')
    plt.title('Top Crypto\'s')
    ax.legend(fontsize = 14)
    
    ################################# Plot 2 ###################################

    if period == 24:
      fig = px.bar(top_crypto_data, x='Name', y='percent_change_24h',
                  hover_data=['Name', 'percent_change_24h','Price', 'Circulating_Supply', 'Trading_Volume', 'volume_24'], 
                  color='Name',
                  labels={'pop':'Crypto Currencies'},
                  height=500, 
                  template = 'ggplot2', title = 'Top Crypto\'s')
    else:
      fig = px.bar(top_crypto_data, x='Name', y='percent_change_7d',
                  hover_data=['Name', 'percent_change_7d','Price', 'Circulating_Supply', 'Trading_Volume', 'volume_24'], 
                  color='Name',
                  labels={'pop':'Crypto Currencies'},
                  height=500, 
                  template = 'ggplot2', title = 'Top Crypto\'s')
    plotly.offline.plot(fig, filename='topCrypto.html')
    ret_Graph('./topCrypto.html')
    fig.show()
    
    ############################################################################

In [8]:
def Growing_Crypto_Investable(crypto_data, amt, period):
    """
    Get the top ```amt``` crypto with maximum growth in the last 24h and plot their graphs

    Parameters
    ----------
    crypto_data : pd.DataFrame
    amt         : int
    period      : int

    Returns
    -------
    Graphs
    """
    ########################### Collect Data Req ###############################

    top_crypto_data = crypto_data.sort_values('percent_change_24h' if period == 24 else 'percent_change_7d', ascending=False).head(amt) 
    
    ################################# Plot 1 ###################################
    f, ax = plt.subplots(figsize=(13,7))

    if period == 24:
      sns.barplot(data=top_crypto_data,
                  x=top_crypto_data['Name'],
                  y=top_crypto_data['percent_change_24h'],
                  palette="gist_earth",#YlOrRd_r
                  label = "percent_change_24h")
    else:
      sns.barplot(data=top_crypto_data,
                  x=top_crypto_data['Name'],
                  y=top_crypto_data['percent_change_7d'],
                  palette="gist_earth",#YlOrRd_r
                  label = "percent_change_7d")
    
    plt.xticks(rotation='vertical')
    plt.title('Top Crypto\'s')
    ax.legend(fontsize = 14)
    
    ################################# Plot 2 ###################################

    if period == 24:
      fig = px.bar(top_crypto_data, x='Name', y='percent_change_24h',
                  hover_data=['Name', 'percent_change_24h','Price', 'Liquidity', 'Circulating_Supply', 'Trading_Volume', 'volume_24'], 
                  color='Name',
                  labels={'pop':'Crypto Currencies'},
                  height=500, 
                  template = 'ggplot2', title = 'Top Crypto\'s')
    else:
      fig = px.bar(top_crypto_data, x='Name', y='percent_change_7d',
                  hover_data=['Name', 'percent_change_7d','Price', 'Liquidity', 'Circulating_Supply', 'Trading_Volume', 'volume_24'], 
                  color='Name',
                  labels={'pop':'Crypto Currencies'},
                  height=500, 
                  template = 'ggplot2', title = 'Top Crypto\'s')
    plotly.offline.plot(fig, filename='bestCrypto.html')
    ret_Graph('./bestCrypto.html')
    fig.show()
    
    ############################################################################

In [9]:
def Analyze_Crypto_Data(liquidable, time):
    """
    Represent the Top Most Promising Cryptos in a 3d scatter plot. 

    Parameters
    ----------
    liquidable : pd.DataFrame 
    time       : int
    
    Returns
    -------
    Graph
    """
    ############################ Get the Data we require #######################
    
    liquidable = liquidable.sort_values('percent_change_24h' if time == 24 else 'percent_change_7d', ascending=False).head(20)
    
    ############################### Plot 1 #####################################
    
    fig = px.scatter_3d(liquidable, 
                        x='Price', 
                        y='Circulating_Supply', 
                        z='percent_change_24h' if time == 24 else 'percent_change_7d',
                        color='Name', 
                        opacity=0.8, 
                        hover_data = ['Name','Price'],
                        size = 'Liquidity', 
                        title = "Most Promising Crypto Currencies",
    )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), template = "plotly_dark")
    plotly.offline.plot(fig, filename='my3Dplot1.html')
    ret_Graph('./my3Dplot1.html')
    fig.show()

    ################################ Plot 2 ####################################
    
    percent_change, Circulating_Supply, Price = liquidable.percent_change_24h if time == 24 else liquidable.percent_change_7d, liquidable.Circulating_Supply, liquidable.Price

    layout = Layout(scene = Scene(
                                                            xaxis = {'title': 'Price'},
                                                            yaxis = {'title': 'Circulating Supply'},
                                                            zaxis = {'title': 'percent_change'}))
    
    fig = go.Figure(data=[go.Scatter3d(
        x=Price,
        y=Circulating_Supply,
        z=percent_change,
        mode='markers',
        marker=dict(
            size=8,
            color=Price,                
            colorscale='Viridis',
            opacity=0.5
        )
    )], layout = layout)
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    plotly.offline.plot(fig, filename='my3Dplot2.html')
    ret_Graph('./my3Dplot2.html')
    fig.show()
    
    ############################################################################

In [9]:
def Analyze_Stocks_Posts(data, parm1, parm2, parm3):
    """
    Plot the effect of social media post using the parameters ```parm1```, ```parm2```, and ```parm3```.
    We use px.scatter_3d() to get the 3-Dimensional Visualization of the growth rate of stocks with 
    respect to the number of occurences / number of upvotes.

    Parameters
    ----------
    data  : pd.DataFrame
    parm1 : str
    parm2 : str
    parm3 : str

    Returns
    -------
    Graph
    """
    ################################# Plot 1 ###################################
    
    fig = px.scatter_3d(data.head(10), 
                        x=parm1, 
                        y=parm2, 
                        z=parm3,
                        color=parm2, 
                        opacity=0.8,
                        hover_data = ['Name','Price'], 
                        title = "Most Promising Stocks",
    )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), template = "plotly_dark")
    fig.show()

    ############################################################################


In [9]:
def Analyze_Crypto_Posts(data, parm1, parm2, parm3):
    """
    Plot the effect of social media post using the parameters ```parm1```, ```parm2```, and ```parm3```.
    We use px.scatter_3d() to get the 3-Dimensional Visualization of the growth rate of cryptos with 
    respect to the number of occurences / number of upvotes.

    Parameters
    ----------
    data  : pd.DataFrame
    parm1 : str
    parm2 : str
    parm3 : str

    Returns
    -------
    Graph
    """
    ################################# Plot 1 ###################################
    
    fig = px.scatter_3d(data.head(10), 
                        x=parm1, 
                        y=parm2, 
                        z=parm3,
                        color=parm2, 
                        opacity=0.8,
                        hover_data = ['Name','Price', 'percent_change_24h'], 
                        title = "Most Promising Crypto Currencies",
    )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), template = "plotly_dark")
    fig.show()

    ############################################################################
